# Space X Data Web Scrapping and Analysis
## 1. Web Scrapping
### Importing Libraries

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

### Functions for Data Extraction

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

### Data Retrieval
#### If we get a 200 success code, data retrievel is success

In [3]:
response = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922")
print(response.status_code)

200


#### Use BeautifulSoup to create objects

In [4]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text,'html.parser')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

####  Extracting tables from the HTML content and identifying the target table for data extraction.

In [5]:
html_tables = soup.find_all('table')

In [6]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

#### Extracting column names from the table headers using BeautifulSoup.

In [7]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
find_th = first_launch_table.find_all('th')
for row in find_th:
    name = extract_column_from_header(row)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


### Data Extraction and Parsing
#### Parsing through table rows and cells to extract relevant data and store it in a dictionary.

In [8]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [9]:
extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table row 
    for rows in table.find_all("tr"):
        # check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # get table element 
        row = rows.find_all('td')
        # if it is number save cells in a dictionary 
        if flag:
            extracted_row += 1
            
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            # Date value
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
              
            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

## 2.Data Cleaning and Preprocessing
### DataFrame Creation

In [10]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,F9 B5B1067.1,Success,3 June 2021,17:29


### Handling Missing Values
#### Checking for missing values in the DataFrame

In [11]:
df.isna().sum()

Flight No.         0
Launch site        0
Payload            0
Payload mass       0
Orbit              0
Customer           2
Launch outcome     0
Version Booster    0
Booster landing    0
Date               0
Time               0
dtype: int64

#### Since only small number (2) of rows contain missing values we drop it

In [12]:
df.dropna(inplace=True)

### Cleaning Data
#### Cleaning and standardizing values in specific columns

In [13]:
df["Launch outcome"].value_counts()

Launch outcome
Success\n    86
Success      32
Failure       1
Name: count, dtype: int64

#### We have to deal with these duplicate success values

In [14]:
df["Launch outcome"] = df["Launch outcome"].replace("Success\n","Success")
df["Launch outcome"].value_counts()

Launch outcome
Success    118
Failure      1
Name: count, dtype: int64

In [15]:
df["Booster landing"].value_counts()

Booster landing
Success         79
No attempt      17
Failure         10
Controlled       5
No attempt\n     4
Uncontrolled     2
Failure          1
Precluded        1
Name: count, dtype: int64

In [16]:
df.loc[df['Booster landing'] == "Failure "]

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
13,14,Cape Canaveral,SpaceX CRS-5,"2,395 kg",LEO,NASA,Success,F9 v1.1,Failure,10 January 2015,09:47


In [17]:
df["Booster landing"] = df["Booster landing"].replace("Failure ","Failure")
df["Booster landing"] = df["Booster landing"].replace("No attempt\n","No attempt")
df["Booster landing"].value_counts()

Booster landing
Success         79
No attempt      21
Failure         11
Controlled       5
Uncontrolled     2
Precluded        1
Name: count, dtype: int64

### Data Type Conversion
#### Converting data types of columns of 'Date', 'Time', and 'Payload mass' for consistency and analysis.

In [18]:
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
df['Time'] = df['Time'].dt.strftime('%H:%M:%S')
df.head()


C:\Users\Induwara\AppData\Local\Temp\ipykernel_6100\135593664.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0B0003.1,Failure,2010-06-04,18:45:00
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,2010-12-08,15:43:00
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt,2012-05-22,07:44:00
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0B0006.1,No attempt,2012-10-08,00:35:00
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0B0007.1,No attempt,2013-03-01,15:10:00


In [19]:
pd.set_option('display.max_rows', None)
df["Payload mass"] = df["Payload mass"].astype(str)
df['Payload mass'].value_counts()

Payload mass
15,600 kg         24
9,600 kg           7
C                  3
0                  2
7,000 kg           2
3,500 kg           2
5,300 kg           2
3,600 kg           2
4,311 kg           2
4,400 kg           1
2,495 kg           1
12,055 kg          1
4,850 kg           1
~4,000 kg          1
2,500 kg           1
3,000 kg           1
7,060 kg           1
5,800 kg           1
7,075 kg           1
2,697 kg           1
5,384 kg           1
6,460 kg           1
13,620 kg          1
2,268 kg           1
4,200 kg           1
14,932 kg          1
~14,000 kg         1
~13,000 kg         1
2,972 kg           1
1,192 kg           1
~12,500 kg         1
3,130 kg           1
~15,440 kg         1
5,000–6,000 kg     1
2,647 kg           1
15,410 kg          1
12,530 kg          1
1,977 kg           1
12,050 kg          1
6,956 kg           1
2,617 kg           1
6,500 kg           1
362 kg             1
4,230 kg           1
6,092 kg           1
4,428 kg           1
2,034 kg           1


In [20]:
df['Payload mass'] = df['Payload mass'].str.replace(' kg', '')
df['Payload mass'] = df['Payload mass'].str.replace('~', '')
df['Payload mass'] = df['Payload mass'].str.replace(',', '')
df['Payload mass'] = df['Payload mass'].str.replace('5000–6000', '5500')
df.drop(df[df['Payload mass'] == "C"].index, inplace = True)
df.drop(df[df['Payload mass'] == "0"].index, inplace = True)
df['Payload mass'].value_counts()


Payload mass
15600    24
9600      7
3600      2
7000      2
3500      2
5300      2
4311      2
2500      1
13620     1
2495      1
12055     1
4850      1
4400      1
525       1
4200      1
3000      1
7060      1
5800      1
7075      1
2697      1
5384      1
6460      1
4000      1
2268      1
2647      1
6500      1
14000     1
13000     1
2972      1
1192      1
12500     1
3130      1
15440     1
14932     1
5500      1
15410     1
12530     1
1977      1
12050     1
6956      1
2617      1
362       1
2150      1
6092      1
5271      1
2034      1
1952      1
1898      1
4159      1
570       1
2395      1
2216      1
4428      1
4535      1
1316      1
2296      1
3325      1
3170      1
500       1
4877      1
553       1
3136      1
4700      1
4696      1
4230      1
2205      1
5200      1
4990      1
475       1
3310      1
6761      1
3669      1
2708      1
6070      1
5600      1
2490      1
4600      1
2257      1
3100      1
3328      1
Name: count, dtype: int64

In [21]:
df["Payload mass"] = df["Payload mass"].astype(int)
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
2,3,CCAFS,Dragon,525,LEO,NASA,Success,F9 v1.0B0005.1,No attempt,2012-05-22,07:44:00
3,4,CCAFS,SpaceX CRS-1,4700,LEO,NASA,Success,F9 v1.0B0006.1,No attempt,2012-10-08,00:35:00
4,5,CCAFS,SpaceX CRS-2,4877,LEO,NASA,Success,F9 v1.0B0007.1,No attempt,2013-03-01,15:10:00
5,6,VAFB,CASSIOPE,500,Polar orbit,MDA,Success,F9 v1.1B1003,Uncontrolled,2013-09-29,16:00:00
6,7,CCAFS,SES-8,3170,GTO,SES,Success,F9 v1.1,No attempt,2013-12-03,22:41:00


In [22]:
df['Version Booster'].value_counts()

Version Booster
F9 B5             26
F9 FT             16
F9 v1.1           13
F9 B5 ♺            9
F9 B4              6
F9 B5B1058.3       1
F9 B5B1049.3       1
F9 B5B1051.2       1
F9 B5B1056.2       1
F9 B5B1047.3       1
F9 B5B1056.3       1
F9 B5B1058.2       1
F9 B5B1049.6       1
F9 B5B1060.2       1
F9 v1.0B0005.1     1
F9 B5B1051.6       1
F9 B5B1048.3       1
F9 B5B1051.8       1
F9 B5B1060.6       1
F9 B5B1061.2       1
F9 B5B1060.7       1
F9 B5B1049.9       1
F9 B5B1051.10      1
F9 B5B1058.8       1
F9 B5B1063.2       1
F9 B5[268]         1
F9 B5B1047.2       1
F9 B5B1049.2       1
F9 B5B1046.3       1
F9 v1.0B0007.1     1
F9 v1.1B1003       1
F9 FT♺             1
F9 FTB1029.2       1
F9 FTB1031.2       1
F9 FTB1035.2       1
F9 FTB1036.2       1
F9 FTB1032.2       1
F9 FTB1038.2       1
F9 B4B1041.2       1
F9 B4B1039.2       1
F9 B5B1046.1       1
F9 B4B1043.2       1
F9 B4B1040.2       1
F9 B4B1045.2       1
F9 B5B1048         1
F9 B5B1046.2       1
F9 B5B1048.2      

In [23]:
df["Version Booster"] = df["Version Booster"].replace("F9 B5 ♺","F9 B5")
df["Version Booster"] = df["Version Booster"].replace("F9 B5[268]","F9 B5")
df["Version Booster"] = df["Version Booster"].replace("F9 FT♺","F9 FT")
df['Version Booster'].value_counts()

Version Booster
F9 B5             36
F9 FT             17
F9 v1.1           13
F9 B4              6
F9 v1.0B0005.1     1
F9 B5B1058.3       1
F9 B5B1049.3       1
F9 B5B1051.2       1
F9 B5B1056.2       1
F9 B5B1047.3       1
F9 B5B1056.3       1
F9 B5B1058.2       1
F9 B5B1049.6       1
F9 B5B1060.2       1
F9 B5B1051.8       1
F9 B5B1051.6       1
F9 B5B1049.2       1
F9 B5B1060.6       1
F9 B5B1061.2       1
F9 B5B1060.7       1
F9 B5B1049.9       1
F9 B5B1051.10      1
F9 B5B1058.8       1
F9 B5B1063.2       1
F9 B5B1048.3       1
F9 B5B1047.2       1
F9 B5B1046.3       1
F9 v1.0B0006.1     1
F9 v1.0B0007.1     1
F9 v1.1B1003       1
F9 FTB1029.2       1
F9 FTB1031.2       1
F9 FTB1035.2       1
F9 FTB1036.2       1
F9 FTB1032.2       1
F9 FTB1038.2       1
F9 B4B1041.2       1
F9 B4B1039.2       1
F9 B5B1046.1       1
F9 B4B1043.2       1
F9 B4B1040.2       1
F9 B4B1045.2       1
F9 B5B1048         1
F9 B5B1046.2       1
F9 B5B1048.2       1
F9 B5B1067.1       1
Name: count, dtype

In [24]:
df.drop(['Flight No.'],axis=1,inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)
df.head()

,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,CCAFS,Dragon,525,LEO,NASA,Success,F9 v1.0B0005.1,No attempt,2012-05-22,07:44:00
1,CCAFS,SpaceX CRS-1,4700,LEO,NASA,Success,F9 v1.0B0006.1,No attempt,2012-10-08,00:35:00
2,CCAFS,SpaceX CRS-2,4877,LEO,NASA,Success,F9 v1.0B0007.1,No attempt,2013-03-01,15:10:00
3,VAFB,CASSIOPE,500,Polar orbit,MDA,Success,F9 v1.1B1003,Uncontrolled,2013-09-29,16:00:00
4,CCAFS,SES-8,3170,GTO,SES,Success,F9 v1.1,No attempt,2013-12-03,22:41:00


### Export the Data

In [25]:
df.to_csv("Data_SpaceX_WebScrapped.csv", index=False)